In [27]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from skimage.feature import hog


car_image_dirs = ["training"]
car_image_locations = glob.glob("./training/vehicles/**/*.png", recursive=True)
non_car_image_locations = glob.glob("./training/non-vehicles/**/*.png",recursive=True)

In [28]:
def read_image(filename):
    image = cv2.imread(filename)
    return cv2.resize(image, (32, 32))

car_images = [read_image(image) for image in car_image_locations]
non_car_images = [read_image(image) for image in non_car_image_locations]

In [31]:
def spatial_color_bin_feature(img,size=(16,16)):
    return np.ravel(cv2.resize(img, size))

def color_hist_feature(img, nbins=32, bins_range=(0, 256)):
    hsv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Compute the histogram of the HSV channels separately
    hhist = np.histogram(hsv_image[:,:,0], bins=nbins, range=bins_range)
    shist = np.histogram(hsv_image[:,:,1], bins=nbins, range=bins_range)
    vhist = np.histogram(hsv_image[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    #bin_edges = hhist[1]
    #bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((hhist[0], shist[0], vhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def hog_feature(img, orient=6, pix_per_cell=8, cell_per_block=2):
    hog_features = []
    for channel in range(img.shape[2]):
      hog_features.append(hog(img[:,:,channel], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=False, feature_vector=True))
    return np.ravel(hog_features)


car_features = [np.concatenate( (color_hist_feature(img), hog_feature(img), spatial_color_bin_feature(img)) ) for img in car_images]
non_car_features = [np.concatenate( (color_hist_feature(img), hog_feature(img), spatial_color_bin_feature(img)) ) for img in non_car_images]


/Users/mccuisto/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [32]:
print(car_features[0].shape)

(1512,)


In [36]:
from sklearn.preprocessing import StandardScaler


X = np.vstack([car_features,non_car_features]).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

y = np.hstack((np.ones(len(car_features)), 
              np.zeros(len(non_car_features))))


In [43]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC

rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

svc = LinearSVC()
svc.fit(X_train, y_train)

print("accuracy:", sum( svc.predict(X_test) == y_test ) / len(y_test))


My SVC predicts:  [ 0.  0.  1.  0.  1.  0.  0.  0.  1.  0.]
For these 10 labels:  [ 0.  0.  1.  0.  1.  0.  0.  0.  1.  1.]
accuracy: 0.985078828829
